In [3]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [6]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import KnapsackToQubo

def knapsack_quadratic_program(values, weights, max_weight):
    # Crie uma instância de QuadraticProgram
    qp = QuadraticProgram()

    # Defina as variáveis binárias para representar a escolha de cada item
    num_items = len(values)
    x = qp.binary_var_list(name="x", length=num_items)

    # Defina a função objetivo para maximizar o valor total dos itens escolhidos
    qp.maximize(linear=[values[i] * x[i] for i in range(num_items)])

    # Adicione a restrição de que o peso total não pode exceder a capacidade máxima
    qp.linear_constraint(linear=[weights[i] * x[i] for i in range(num_items)], sense="LE", rhs=max_weight, name="weight_constraint")

    kqp = prob.to_quadratic_program()
    return prob, kqp

# Dados do problema da mochila
values = [10, 13, 18, 31, 7]
weights = [2, 4, 6, 7, 3]
max_weight = 10

# Criar o problema de otimização quadrática (QUBO) correspondente
qubo = knapsack_quadratic_program(values, weights, max_weight)

# Agora você pode resolver o QUBO usando o QAOA ou outro algoritmo quântico


Traceback (most recent call last):
  Cell In[6], line 2
    from qiskit_optimization.converters import KnapsackToQubo
ImportError: cannot import name 'KnapsackToQubo' from 'qiskit_optimization.converters' (/opt/conda/lib/python3.10/site-packages/qiskit_optimization/converters/__init__.py)

Use %tb to get the full traceback.


In [15]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA
import numpy as np

def knapsack_quadratic_program():
    # Put values, weights and max_weight parameter for the Knapsack()
    values = [10, 13, 18, 31, 7]
    weights = [2, 4, 6, 7, 3]
    max_weight = 10
    ##############################
    # Provide your code here
    
    
    prob = Knapsack(values, weights, max_weight)
    
    ##############################
    # to_quadratic_program generates a corresponding QuadraticProgram of the instance of the knapsack problem.
    kqp = prob.to_quadratic_program()
    return prob, kqp

prob,quadratic_program=knapsack_quadratic_program()
quadratic_program

<QuadraticProgram: maximize 10*x_0 + 13*x_1 + 18*x_2 + 31*x_3 + 7*x_4, 5 variables, 1 constraints, 'Knapsack'>

We can solve the problem using the classical NumPyMinimumEigensolver to find the minimum eigenvector, which may be useful as a reference without doing things by Dynamic Programming; we can also apply QAOA.

In [16]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 

result:
 fval=41.0, x_0=1.0, x_1=0.0, x_2=0.0, x_3=1.0, x_4=0.0, status=SUCCESS

 index of the chosen items: [0, 3]


In [20]:
# QAOA
from qiskit.primitives import Sampler
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.algorithms.optimizers import COBYLA

seed = 123
algorithm_globals.random_seed = seed
sampler = Sampler()
aer_sampler = AerSampler(run_options={"shots": 1000, "seed": seed})

optimizer = COBYLA()

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(aer_sampler, optimizer, reps=2))
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 


result:
 fval=41.0, x_0=1.0, x_1=0.0, x_2=0.0, x_3=1.0, x_4=0.0, status=SUCCESS

 index of the chosen items: [0, 3]


In [22]:
def knapsack_quadratic_program(values, weights, max_weight):
    # Crie uma instância de QuadraticProgram
    qp = QuadraticProgram()

    # Defina as variáveis binárias para representar a escolha de cada item
    num_items = len(values)
    x = qp.binary_var_list(name="x", length=num_items)

    # Defina a função objetivo para maximizar o valor total dos itens escolhidos
    qp.maximize(linear=[values[i] * x[i] for i in range(num_items)])

    # Adicione a restrição de que o peso total não pode exceder a capacidade máxima
    qp.linear_constraint(linear=[weights[i] * x[i] for i in range(num_items)], sense="LE", rhs=max_weight, name="weight_constraint")

    return qp

# Dados do problema da mochila
values = [10, 13, 18, 31, 7]
weights = [2, 4, 6, 7, 3]
max_weight = 10

# Criar o problema de otimização quadrática (QUBO) correspondente
qubo = knapsack_quadratic_program(values, weights, max_weight)

Traceback (most recent call last):
  Cell In[22], line 23
    qubo = knapsack_quadratic_program(values, weights, max_weight)
  Cell In[22], line 7 in knapsack_quadratic_program
    x = qp.binary_var_list(name="x", length=num_items)
TypeError: QuadraticProgram.binary_var_list() got an unexpected keyword argument 'length'

Use %tb to get the full traceback.
